In [1]:
%load_ext autoreload
%aimport gempipe, gempipe.interface, gempipe.interface.sanity, gempipe.interface.gaps, gempipe.interface.medium
%autoreload 1

## _Tutorial:_ multi-strain analysis

To do...

In [5]:
import gempipe

# initialize gempipe on the 'gempipe recon' --outdir:
panmodel = gempipe.initialize("tutoring_materials/plantarum")

Loading PAM (tutoring_materials/plantarum/pam.csv)...
Loading functional annotation table (tutoring_materials/plantarum/annotation.csv)...
Loading report table (tutoring_materials/plantarum/report.csv)...
Loading draft pan-GSMM (tutoring_materials/plantarum/draft_panmodel.json)...


We start from setting the unconstrained lower and upper bounds. Originally they were set as -999999/999999, and this doesn't represent an issue. Anyway, we prefer to have them in the style -1000/1000, that seems more established today. We use the function [gempipe.reset_unconstrained_bounds](https://gempipe.readthedocs.io/en/latest/autoapi/gempipe/interface/sanity/index.html#gempipe.interface.sanity.reset_unconstrained_bounds):